# Wage vs CPI Trend in Alberta

This visualization uses the **Silver layer** tables:

- `wages_silver` (average wages over time)
- `cpi_silver` (consumer price index over time)

The goal is to show how wages and inflation move together over time for Alberta,
as a first step toward building an **Affordability Index** in the Gold layer.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW wage_cpi_trend AS
SELECT
  w.date,
  w.geo,
  w.avg_wage,
  c.cpi_value
FROM mycatalog.default.wages_silver w
JOIN mycatalog.default.cpi_silver c
  ON w.date = c.date
  AND w.geo = c.geo
ORDER BY w.date;



In [0]:
%sql
SELECT *
FROM wage_cpi_trend
LIMIT 20;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
  year(date) AS year,
  month(date) AS month,
  AVG(avg_wage) AS avg_wage
FROM mycatalog.default.wages_silver
GROUP BY year, month
ORDER BY year, month;


Databricks visualization. Run in Databricks to view.

In [0]:
import matplotlib.pyplot as plt

df = spark.table("mycatalog.default.wages_silver").toPandas()

df = df.sort_values("date")

plt.figure(figsize=(10, 5))
plt.plot(df["date"], df["avg_wage"])
plt.xlabel("Date")
plt.ylabel("Average Wage")
plt.title("Average Wage Trend in Alberta")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [0]:
%sql
SELECT *
FROM wage_cpi_trend


In [0]:
%sql
SELECT 
    date,
    AVG(avg_wage) AS avg_wage,
    AVG(cpi_value) AS cpi_value
FROM wage_cpi_trend
GROUP BY date
ORDER BY date;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
    date,
    avg_wage,
    avg_wage - LAG(avg_wage, 12) OVER (ORDER BY date) AS rolling_12m_change
FROM mycatalog.default.wages_silver
ORDER BY date;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW wage_cpi_rolling_wages AS
SELECT
    date,
    avg_wage AS rolling_12m_change
FROM mycatalog.default.wages_silver
ORDER BY date;

In [0]:
%sql
SELECT
    date,
    rolling_12m_change,
    AVG(rolling_12m_change)
        OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
        AS smoothed
FROM wage_cpi_rolling_wages
ORDER BY date;

In [0]:
%sql
WITH wages AS (
  SELECT
      date,
      avg_wage - LAG(avg_wage, 12) OVER (ORDER BY date) AS wage_change_12m
  FROM mycatalog.default.wages_silver
),
cpi AS (
  SELECT
      date,
      cpi_value - LAG(cpi_value, 12) OVER (ORDER BY date) AS cpi_change_12m
  FROM mycatalog.default.cpi_silver
)

SELECT
    w.date,
    w.wage_change_12m,
    c.cpi_change_12m
FROM wages w
JOIN cpi c
  ON w.date = c.date
WHERE w.wage_change_12m IS NOT NULL
  AND c.cpi_change_12m IS NOT NULL
ORDER BY date;


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.